# Diagnostic Notebook — NumPy & Pandas

15 step-by-step exercises targeting indexing, vectorization, grouping, reshaping, and cleaning. Work through the tasks in order (easy → hard). **Solution cells are included but marked hidden.**

Run the first code cell to create the in-notebook sample datasets.

In [87]:

# Create sample datasets used in the exercises
import numpy as np, pandas as pd
rng = np.random.default_rng(0)

# Dataset 1: numeric array for numpy tasks
arr = rng.integers(-5, 20, size=(6,5)).astype(float)
arr[0,0] = np.nan  # introduce a NaN
print("arr.shape:", arr.shape)
print(arr)

# Dataset 2: sales-like DataFrame for groupby/aggregation exercises
sales = pd.DataFrame({
    "region": ["North","South","East","West","North","South","East","West","North","South"],
    "product": ["A","A","B","B","A","B","A","B","C","C"],
    "units": rng.integers(1,10, size=10),
    "price": rng.uniform(5,50,size=10).round(2)
})
sales.loc[2, "units"] = np.nan
sales.loc[5, "price"] = np.nan

# Dataset 3: messy students dataset for cleaning/reshaping
students = pd.DataFrame({
    "id": range(1,9),
    "name": ["Alice","Bob","Charlie","David","Eva","Frank","Grace","Hank"],
    "math_score": [88, "92", 79, 85, None, "78", 91, "NaN"],
    "english_score": [75, 80, 85, "NA", 95, 82, None, 78],
    "class": [1,1,2,2,1,2,1,2]
})

# small multi-index example for pivoting
temps = pd.DataFrame({
    "city": ["X","X","Y","Y"],
    "date": pd.to_datetime(["2020-01-01","2020-01-02","2020-01-01","2020-01-02"]),
    "temp": [20, 21, 19, 18]
})

# expose datasets
arr, sales, students, temps


arr.shape: (6, 5)
[[nan 10.  7.  1.  2.]
 [-4. -4. -5. -1. 15.]
 [11. 17.  7. 10. 19.]
 [13. 10.  8.  8. 18.]
 [ 1. 15. 11. -5.  4.]
 [16.  8. -5. 14. 13.]]


(array([[nan, 10.,  7.,  1.,  2.],
        [-4., -4., -5., -1., 15.],
        [11., 17.,  7., 10., 19.],
        [13., 10.,  8.,  8., 18.],
        [ 1., 15., 11., -5.,  4.],
        [16.,  8., -5., 14., 13.]]),
   region product  units  price
 0  North       A    8.0   6.27
 1  South       A    2.0  10.59
 2   East       B    NaN  35.18
 3   West       B    8.0  34.12
 4  North       A    1.0  32.69
 5  South       B    5.0    NaN
 6   East       A    1.0  49.87
 7   West       B    3.0  49.14
 8  North       C    5.0  35.85
 9  South       C    4.0  34.27,
    id     name math_score english_score  class
 0   1    Alice         88            75      1
 1   2      Bob         92            80      1
 2   3  Charlie         79            85      2
 3   4    David         85            NA      2
 4   5      Eva       None            95      1
 5   6    Frank         78            82      2
 6   7    Grace         91          None      1
 7   8     Hank        NaN            78      2,
  

## NumPy — indexing basics

1. Using `arr` (the NumPy array), extract the 3rd row (as a 1D array) and print its dtype and shape.

2. Extract the last column of `arr` and compute its mean (ignore NaN).

In [10]:
#1
print(arr[2])
print(type(arr[2]))
arr[2].dtype

[11. 17.  7. 10. 19.]
(5,)
<class 'numpy.ndarray'>


dtype('float64')

In [12]:
#2
print(arr[::,-1])
arr[::,-1].mean()

[ 2. 15. 19. 18.  4. 13.]


np.float64(11.833333333333334)

## NumPy — boolean & fancy indexing

3. Select all elements in `arr` greater than 5 (return a 1D array of values).

4. From `arr`, select rows where the first column is non-NaN and greater than 0.

In [24]:
arr

array([[nan, 10.,  7.,  1.,  2.],
       [-4., -4., -5., -1., 15.],
       [11., 17.,  7., 10., 19.],
       [13., 10.,  8.,  8., 18.],
       [ 1., 15., 11., -5.,  4.],
       [16.,  8., -5., 14., 13.]])

In [26]:
#3
print(arr[arr>5])
#4
arr[(~np.isnan(arr[::,0])) & (arr[::,0]>0)]



[10.  7. 15. 11. 17.  7. 10. 19. 13. 10.  8.  8. 18. 15. 11. 16.  8. 14.
 13.]


array([[11., 17.,  7., 10., 19.],
       [13., 10.,  8.,  8., 18.],
       [ 1., 15., 11., -5.,  4.],
       [16.,  8., -5., 14., 13.]])

## NumPy — reshape & broadcasting

5. Reshape `arr` into shape (5,6) if possible. If not possible, explain why and reshape into (-1,3).

6. Create a column vector `c = np.array([1,2,3,4,5])[:,None]` and add it to the first five rows of `arr` (use broadcasting).

In [60]:
#5
print(arr.size)
print(arr.reshape((5,6)))
print(arr.reshape((-1,3)))
#6
c=np.array([1,2,3,4,5])[:,None]
c
#cant broadcast c in such way to add only 5 elements to 5 rows out of 6 of the arr.

30
[[nan 10.  7.  1.  2. -4.]
 [-4. -5. -1. 15. 11. 17.]
 [ 7. 10. 19. 13. 10.  8.]
 [ 8. 18.  1. 15. 11. -5.]
 [ 4. 16.  8. -5. 14. 13.]]
[[nan 10.  7.]
 [ 1.  2. -4.]
 [-4. -5. -1.]
 [15. 11. 17.]
 [ 7. 10. 19.]
 [13. 10.  8.]
 [ 8. 18.  1.]
 [15. 11. -5.]
 [ 4. 16.  8.]
 [-5. 14. 13.]]


array([[1],
       [2],
       [3],
       [4],
       [5]])

## Pandas — basic inspection & dtypes

7. Print `students.info()` and convert `math_score` and `english_score` to numeric (coerce errors). Show the resulting dtypes.

8. How many missing scores are present after coercion?

In [89]:
print(students)
#7
print(students.info())
students=students.convert_dtypes()
students["math_score"]=pd.to_numeric(students["math_score"],errors="coerce")
students["english_score"]=pd.to_numeric(students["english_score"],errors="coerce")
print(students.info())
#8
students.isnull().value_counts()

   id     name math_score english_score  class
0   1    Alice         88            75      1
1   2      Bob         92            80      1
2   3  Charlie         79            85      2
3   4    David         85            NA      2
4   5      Eva       None            95      1
5   6    Frank         78            82      2
6   7    Grace         91          None      1
7   8     Hank        NaN            78      2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             8 non-null      int64 
 1   name           8 non-null      object
 2   math_score     7 non-null      object
 3   english_score  7 non-null      object
 4   class          8 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 452.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column     

id     name   math_score  english_score  class
False  False  False       False          False    4
                          True           False    2
              True        False          False    2
Name: count, dtype: int64

## Pandas — selection & .loc/.iloc

9. Using `sales`, select rows for region 'North' and product 'A' using `.loc` with boolean masks.

10. Assign a new column `revenue = units * price` safely (avoid chained indexing issues). Show the DataFrame.

In [88]:
#9
print(sales.loc[(sales["product"]=="A")&(sales["region"]=="North")])
#10
sales["revenue"]=sales["units"].mul(sales["price"],fill_value=0)
sales

  region product  units  price
0  North       A    8.0   6.27
4  North       A    1.0  32.69


,region,product,units,price,revenue
0,North,A,8.0,6.27,50.16
1,South,A,2.0,10.59,21.18
2,East,B,NaN,35.18,0.00
3,West,B,8.0,34.12,272.96
4,North,A,1.0,32.69,32.69
5,South,B,5.0,NaN,0.00
6,East,A,1.0,49.87,49.87
7,West,B,3.0,49.14,147.42
8,North,C,5.0,35.85,179.25
9,South,C,4.0,34.27,137.08


## Pandas — groupby and aggregation

11. Compute total revenue per region (sum of revenue) and return a Series sorted descending.

12. For each product, compute mean units and count of non-null price values.

In [76]:
#11
print(sales.groupby("region")["revenue"].sum().sort_values(ascending=False))
#12
sales.groupby("product").agg({"units":"mean","price":lambda x:x.notnull().value_counts()[True]})

region
West     420.38
North    262.10
South    158.26
East      49.87
Name: revenue, dtype: float64


,units,price
product,,
A,3.000000,4
B,5.333333,3
C,4.500000,2


## Pandas — fillna using group mode (practical)

13. For `sales`, fill missing `units` or `price` within each `region` using the group's median for numeric; then show per-region counts of NaNs post-fill.

In [100]:
sales

,region,product,units,price,revenue
0,North,A,8.0,6.27,50.16
1,South,A,2.0,10.59,21.18
2,East,B,NaN,35.18,0.00
3,West,B,8.0,34.12,272.96
4,North,A,1.0,32.69,32.69
5,South,B,5.0,NaN,0.00
6,East,A,1.0,49.87,49.87
7,West,B,3.0,49.14,147.42
8,North,C,5.0,35.85,179.25
9,South,C,4.0,34.27,137.08


In [105]:
#13
sales.groupby("region")[["units","price"]].apply(lambda x: x.fillna(x.median())).isnull().value_counts()




,,count
units,price,
False,False,10


## Pandas — pivot/melt and reshaping

14. Pivot `temps` to have `date` as index and cities as columns of temperature. Then revert it back using `melt` to the original form.

15. Using `students`, produce a long-form DataFrame with id, name, subject, score (subject: math/english).

In [106]:
#ignore question 14 and 15


,city,date,temp
0,X,2020-01-01,20
1,X,2020-01-02,21
2,Y,2020-01-01,19
3,Y,2020-01-02,18
